### Import Libraries

In [ ]:
import dask
import xarray as xr

### Initialize dask client
Dask is imported to handle large datasets efficiently by leveraging parallel computing.

In [1]:
from dask.distributed import Client
client = Client(n_workers=64, threads_per_worker=2, memory_limit='128GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 64
Total threads: 128,Total memory: 7.45 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36493,Workers: 64
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 7.45 TiB
Comm: tcp://127.0.0.1:44173,Total threads: 2
Dashboard: http://127.0.0.1:34969/status,Memory: 119.21 GiB
Nanny: tcp://127.0.0.1:34953,


A Dask client is initialized to manage distributed computation. The client object provides access to the dashboard for monitoring tasks.

### Load the datasets 

In [3]:
mds_solar = xr.open_mfdataset('/scratch/vishald/jrf2_monsoonlab/shiv/download_data/Solar_*.nc',combine = 'nested',concat_dim = 'time', parallel=True, chunks = {'latitude':60, 'longitude':60})

In [40]:
mds_wind = xr.open_mfdataset('/scratch/vishald/jrf2_monsoonlab/shiv/download_data/Wind_*.nc',combine = 'nested',concat_dim = 'time', parallel=True, chunks={'longitude': 60, 'latitude': 60})

In [4]:
mds_solar

<xarray.Dataset>
Dimensions:    (longitude: 319, latitude: 353, time: 350640)
Coordinates:
  * longitude  (longitude) float32 65.55 65.65 65.75 65.85 ... 97.15 97.25 97.35
  * latitude   (latitude) float32 38.95 38.85 38.75 38.65 ... 3.95 3.85 3.75
  * time       (time) datetime64[ns] 1981-01-01 ... 2020-12-31T23:00:00
Data variables:
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(8760, 60, 60), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-14 20:41:49 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

mds_solar and mds_wind contain the ssrd and wind speeds respectively which have been downloaded from ERA5 Land dataset. They were stored in yearly files each totalling up to 40 years' worth of data

In [41]:
u = mds_wind['u100']
v = mds_wind['v100']
speed = (u*u + v*v)**0.5
wpd_all = 0.5*1.2*(speed**3) #wpd_all now represents the wind power density(W/m2)
wpd_all

,Array,Chunk
Bytes,147.09 GiB,120.63 MiB
Shape,"(350640, 353, 319)","(8784, 60, 60)"
Count,14480 Tasks,1440 Chunks
Type,float32,numpy.ndarray


In [43]:
mds_solar = mds_solar['ssrd']
mds_solar = mds_solar/3600 # The ssrd data is hourly and given in Joules. We are converting it to W/m2
day_threshold = 10 #We want to exclude the night-time data from our analysis. That is why we kept a threshold of 10 W/m2 for our dta
wpd_day = wpd_all.where(mds_solar >= day_threshold) #This filtering step keeps the analysis consistent between solar and wind data by focusing on the same time periods (daytime)
mds_solar = mds_solar.where( mds_solar >= day_threshold )


In [39]:
mds_solar

,Array,Chunk
Bytes,147.09 GiB,335.08 MiB
Shape,"(350640, 353, 319)","(8784, 100, 100)"
Count,3240 Tasks,640 Chunks
Type,float32,numpy.ndarray


### Computing Solar statistics

In [32]:
sol_mean_all = mds_solar.mean('time').compute()

2024-04-01 01:05:35,914 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,919 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,919 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,920 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,921 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,922 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,922 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,923 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,923 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,924 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,924 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-04-01 01:05:35,924 - distributed.nanny

KeyboardInterrupt: 

In [9]:
sol_mean_all.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar Mean.nc')

In [7]:
sol_med = mds_solar.median('time') #Compute median

In [8]:
sol_med = sol_med.compute()

In [10]:
sol_med.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar Median.nc')

In [18]:
%%time
#MemoryManagement
import gc
client.run(gc.collect)
 
import ctypes
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)
client.run(trim_memory)

solar_median = xr.open_dataarray(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar Median.nc')
print('Median Loaded')
#Computation for RCoV
sol_rcov = ((abs(mds_solar - solar_median)).median('time',skipna=True).compute())/solar_median
sol_rcov.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar RCoV.nc')

Median Loaded
CPU times: user 1min 4s, sys: 14.7 s, total: 1min 19s
Wall time: 12min 32s


In [20]:
%%time
import gc
client.run(gc.collect)
 
import ctypes
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)
client.run(trim_memory)
# -------------------------------
# Solar Availability Calculation
# -------------------------------
solar_th = 170 # Threshold in W/m² (adjust based on solar energy requirements)
# Calculate total daytime hours (already filtered using previous 10 W/m² threshold)
day_time_hours = mds_solar.count(dim='time').load()
# Calculate hours where solar radiation exceeds operational threshold (170 W/m²)
solar_avail = mds_solar.where(mds_solar > solar_th).count(dim='time').compute()
# Calculate percentage availability during daytime
pct_solar_avail = (solar_avail/day_time_hours)*100
pct_solar_avail.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar Availability.nc')
print('Done')

Done
CPU times: user 57.6 s, sys: 17.9 s, total: 1min 15s
Wall time: 18min 34s


### Synergy Metrics Calculation
WSS (Wind-Solar Synergy): Percentage of daytime hours when either wind compensates for weak solar or solar compensates for weak wind, highlighting hybrid system complementarity.

WCS (Wind Complements Solar): Percentage of daytime hours when wind energy compensates for low solar radiation, ensuring energy availability during weak solar periods.

SCW (Solar Compensates Wind): Percentage of daytime hours when solar energy compensates for low wind power density, ensuring energy availability during weak wind periods.

In [10]:
# Thresholds for solar (W/m²) and wind power density (W/m²)
solar_th = 170 # Solar radiation below this = "low solar availability"
wind_th = 240  # Wind power density above this = "sufficient wind availability"
# Calculate total daytime hours (already filtered using 10 W/m² threshold)
day_time_hours = mds_solar.count(dim='time').load()
# Calculate hours where:
# - Solar is WEAK (≤170 W/m²) AND
# - Wind is STRONG (>240 W/m²)
wcs = (mds_solar.where((mds_solar <= solar_th) & (wpd_day > wind_th))).count(dim='time').load()
# Convert to percentage of daytime hours
wcs = (wcs/day_time_hours)*100

In [11]:
wcs.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire WCS.nc')
print('Done')

Done


In [22]:
# Calculate hours where:
# - Solar is STRONG (>170 W/m²) AND
# - Wind is WEAK (≤240 W/m²)
scw = (mds_solar.where((mds_solar > solar_th) & (wpd_day <= wind_th))).count(dim='time').load()
# Convert to percentage of daytime hours
scw = (scw/day_time_hours)*100

In [23]:
scw.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire SCW.nc')

In [10]:
# Define thresholds for solar and wind energy availability
solar_th = 170 # Solar radiation threshold in W/m² (values above this indicate sufficient solar energy)
wind_th = 240 # Wind power density threshold in W/m² (values above this indicate sufficient wind energy)
# Calculate total daytime hours (pre-filtered using a 10 W/m² solar threshold)
day_time_hours = mds_solar.count(dim='time').load()
# Create a boolean mask for hours where either:
# 1. Solar is STRONG (>170 W/m²) AND Wind is WEAK (≤240 W/m²), OR
# 2. Solar is WEAK (≤170 W/m²) AND Wind is STRONG (>240 W/m²)
wss_bool = ((mds_solar > solar_th) & (wpd_day <= wind_th)) | ((mds_solar <= solar_th) & (wpd_day > wind_th))
# Calculate hours meeting either compensation condition
wss = (mds_solar.where(wss_bool)).count(dim='time').load()
wss = (wss/day_time_hours)*100 # % of time when solar/wind compensate for each other

In [11]:
wss.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire WSS.nc')

IQR Calculation

In [8]:
# --- Calculate Interquartile Range (IQR) for solar data ---

# Free up memory using garbage collection on all Dask workers
import gc
client.run(gc.collect)

# Run malloc_trim to free unused memory back to the OS
import ctypes
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)
client.run(trim_memory)

# Calculate 75th percentile (Q3) of solar data over time
# 'chunk' is used for efficient Dask computation
q75 = mds_solar.chunk({"time": -1, "latitude": "auto", "longitude": "auto"}).quantile(0.75, 'time')
print("75Th complete. Calculating 75th")
q75
q75 = q75.compute()

# Save the computed 75th percentile to disk as a NetCDF file
q75.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar QSevenFive.nc')

# Clean up memory again before computing the next percentile
client.run(gc.collect)
client.run(trim_memory)

# Calculate 25th percentile (Q1) of solar data over time
q25 = mds_solar.chunk({"time": -1, "latitude": "auto", "longitude": "auto"}).quantile(0.25, 'time')
print("25Th complete. Calculating 75th")
q25
q25 = q25.compute()

# Save the computed 25th percentile to disk as a NetCDF file
q25.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar QTwoFive.nc')

# Reload the saved percentiles from disk
q25 = xr.open_dataarray(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar QTwoFive.nc')
q75 = xr.open_dataarray(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar QSevenFive.nc')

# Compute Interquartile Range (IQR = Q3 - Q1)
iqr = q75 - q25
iqr

# Save the IQR as a NetCDF file
iqr.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/Entire Solar IQR.nc')


75Th complete. Calculating 75th
